## Extracting lobe junctions as fissures from Ottawa chest CT dataset (https://zenodo.org/records/12690803) (with all 5 lobes available and 2mm or lesser slice thickness) and Stanford datasets 

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_dilation
from skimage.morphology import skeletonize

# ==== INPUTS ====
imagesTr_dir = r"--- INSERT PATH ---"   # derive case IDs from <ID>_0000.nii.gz
labelsTr     = r"--- INSERT PATH ---"   # existing labels (to define 'missing'); can be empty if processing all
lobes_dir    = r"--- INSERT PATH ---"   # <ID>.nii.gz (lobe labels: 1=RUL, 2=RML, 3=RLL, 4=LUL, 5=LLL)
output_dir   = r"--- INSERT PATH ---"   # output <ID>_RHF.nii.gz, <ID>_ROF.nii.gz, <ID>_LOF.nii.gz
os.makedirs(output_dir, exist_ok=True)

# ==== helpers ====
def find_junction(mask_a: np.ndarray, mask_b: np.ndarray) -> np.ndarray:
    """Shared boundary (1-pixel band) between two binary masks on a 2D slice."""
    edge_a = binary_dilation(mask_a) & mask_b
    edge_b = binary_dilation(mask_b) & mask_a
    return edge_a | edge_b

def reconnect_broken_junctions(junction_2d: np.ndarray, dilation_size: int = 2) -> np.ndarray:
    """Skeletonize → endpoint padding → skeletonize again; returns boolean."""
    skel = skeletonize(junction_2d.astype(bool))
    endpoint_img = np.zeros_like(skel, dtype=np.uint8)
    H, W = skel.shape
    for i in range(dilation_size, H - dilation_size):
        for j in range(dilation_size, W - dilation_size):
            if skel[i, j]:
                neighbors = skel[i-1:i+2, j-1:j+2].sum() - 1
                if neighbors == 1:
                    endpoint_img[i-dilation_size:i+dilation_size+1,
                                 j-dilation_size:j+dilation_size+1] = 1
    repaired = skeletonize((skel | endpoint_img).astype(bool))
    return repaired

def repair_and_thicken(junc2d: np.ndarray) -> np.ndarray:
    """Two rounds of repair + light dilation for continuity."""
    r1 = reconnect_broken_junctions(junc2d)
    t1 = binary_dilation(r1, iterations=1)
    r2 = reconnect_broken_junctions(t1)
    t2 = binary_dilation(r2, iterations=1)
    return (t2 > 0)

# ==== choose which IDs to process ====
image_ids = [f.replace('_0000.nii.gz', '') for f in os.listdir(imagesTr_dir) if f.endswith('_0000.nii.gz')]
label_ids = [f.replace('.nii.gz', '')      for f in os.listdir(labelsTr)     if f.endswith('.nii.gz')]
case_ids  = [cid for cid in image_ids if cid not in label_ids]   # process "missing" relative to labelsTr
# If you prefer to process ALL with lobes present, uncomment:
# case_ids = [f.replace('.nii.gz','') for f in os.listdir(lobes_dir) if f.endswith('.nii.gz')]

for case_id in case_ids:
    lobe_path = os.path.join(lobes_dir, f"{case_id}.nii.gz")
    if not os.path.isfile(lobe_path):
        print(f"❌ Missing lobe file: {lobe_path}")
        continue

    img  = nib.load(lobe_path)
    data = np.rint(img.get_fdata()).astype(np.int32)  # ensure integer labels

    # Binary outputs (0/1) for each fissure
    rhf = np.zeros_like(data, dtype=np.uint8)  # Right Horizontal: 1↔2
    rof = np.zeros_like(data, dtype=np.uint8)  # Right Oblique: (1↔3) ∪ (2↔3)
    lof = np.zeros_like(data, dtype=np.uint8)  # Left Oblique: 4↔5

    # === CORONAL slices: iterate over axis-1, slice as data[:, y, :] ===
    for y in range(data.shape[1]):
        sl = data[:, y, :]

        # RHF (1 ↔ 2)
        j_rhf = find_junction(sl == 1, sl == 2)
        if j_rhf.any():
            rhf[:, y, :] = repair_and_thicken(j_rhf).astype(np.uint8)

        # ROF = union of (1 ↔ 3) and (2 ↔ 3)
        j_rof_13 = find_junction(sl == 1, sl == 3)
        j_rof_23 = find_junction(sl == 2, sl == 3)
        j_rof = (j_rof_13 | j_rof_23)
        if j_rof.any():
            rof[:, y, :] = repair_and_thicken(j_rof).astype(np.uint8)

        # LOF (4 ↔ 5)
        j_lof = find_junction(sl == 4, sl == 5)
        if j_lof.any():
            lof[:, y, :] = repair_and_thicken(j_lof).astype(np.uint8)

    # === Save as uint8 (0/1) ===
    nib.save(nib.Nifti1Image(rhf.astype(np.uint8), img.affine, img.header), os.path.join(output_dir, f"{case_id}_RHF.nii.gz"))
    nib.save(nib.Nifti1Image(rof.astype(np.uint8), img.affine, img.header), os.path.join(output_dir, f"{case_id}_ROF.nii.gz"))
    nib.save(nib.Nifti1Image(lof.astype(np.uint8), img.affine, img.header), os.path.join(output_dir, f"{case_id}_LOF.nii.gz"))

    print(f"✅ Saved coronal fissures for {case_id}: RHF/ROF/LOF")
